Import the required libraries

In [1]:
# pip install nltk

In [2]:
import nltk

In [3]:
import os
import sys
import pickle
from collections import namedtuple
from datetime import datetime
import numpy as np
import apache_beam as beam
from apache_beam.transforms import util
import tensorflow as tf
import tensorflow_hub as hub
import annoy
import pandas as pd
from nltk.tokenize import sent_tokenize , word_tokenize
# from sklearn.random_projection import gaussian_random_matrix

In [4]:
print('TF version: {}'.format(tf.__version__))
print('TF-Hub version: {}'.format(hub.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

TF version: 2.5.0
TF-Hub version: 0.12.0
Apache Beam version: 2.31.0


In [5]:
file = pd.read_csv("xgimiMoGoPro_with_dates_allv02.csv")
file.head()

,product,title,date,rating,body
0,"Xgimi Mogo Pro, Mini Portable Projector Smart,...","Best projector comparing to ViewSonic M1+ , Ne...","Reviewed in the United States on March 8, 2020",5.0,"I tried ViewSonic M1+ , Nebula capsule max, ca..."
1,"Xgimi Mogo Pro, Mini Portable Projector Smart,...",Very nice Product,"Reviewed in the United States on November 25, ...",5.0,Quick summary- this device provides high quali...
2,"Xgimi Mogo Pro, Mini Portable Projector Smart,...",WOW... AMAZING projector for its size 🥰,"Reviewed in the United States on January 6, 2021",5.0,I currently have Towond 6500LUX projector and ...
3,"Xgimi Mogo Pro, Mini Portable Projector Smart,...",Not 1080p + Not bright enough + no Netflix,"Reviewed in the United States on January 9, 2021",3.0,"I agree with a lot of reviews, it’s very porta..."
4,"Xgimi Mogo Pro, Mini Portable Projector Smart,...","Good projector: Portable, good resolution, and...","Reviewed in the United States on May 11, 2020",5.0,XGIMI MoGo pro is a portable projector with a ...


In [6]:
reviews=file["body"]
type(reviews)

pandas.core.series.Series

In [7]:
import re
def de_emojify(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [50]:
file = []

for rev in reviews:
    text = rev.replace("”",'” ')
    text = text.replace('''[+]''',"")
    text = text.replace('''[-]''',"")
    text = de_emojify(text)
#     print(text)
    sents = sent_tokenize(text)
    for line in sents:
        file.append(line)

In [9]:
type(file)

list

In [10]:
print("Number of reviews: ",len(reviews))
print("Number of sentences: ",len(file))

Number of reviews:  170
Number of sentences:  1087


In [11]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
pd.DataFrame(file).to_csv("XGlilnes.csv", header=None)

In [13]:
file = pd.read_csv("XGlilnes.csv", header=None)
file.head()

,0,1
0,0,"I tried ViewSonic M1+ , Nebula capsule max, ca..."
1,1,Below are the pros and consPros:1) Full HD2) D...
2,2,But you can watch using XTV manager.2) No offi...
3,3,But you can cast using Chromecast.I hope they ...
4,4,Quick summary- this device provides high quali...


In [14]:
file = file[1]
type(file)

pandas.core.series.Series

In [15]:
file

0       I tried ViewSonic M1+ , Nebula capsule max, ca...
1       Below are the pros and consPros:1) Full HD2) D...
2       But you can watch using XTV manager.2) No offi...
3       But you can cast using Chromecast.I hope they ...
4       Quick summary- this device provides high quali...
                              ...                        
1082    This projector comes with a very intuitive set...
1083    Works well with external HDMI connections like...
1084    The volume setting interface can be made bette...
1085    But over all my roommates and I really enjoy t...
1086    The high definition movie feels really good ma...
Name: 1, Length: 1087, dtype: object

In [16]:
# from nltk.tokenize import sent_tokenize , word_tokenize
# import re
# sents=[]
# for review in file:
#     review = review.lower()
# #     review = re.sub(r'\S*@\S*\s*', '', review)
# #     review = review.translate(str.maketrans('', '', string.punctuation))
# #     review = review.split('\n')
    
#     sentences = sent_tokenize(review)
    
    
#     for sentence in review:
# #         print(sentence)
# #         tokens = word_tokenize(sentence)
# #         tokens = [word for word in tokens if (word not in string.punctuation)]
# #         sentence =  " ".join(tokens)
# #         print(sentence)
#         sents.append(sentence)
        
with open('xgimitext.txt', 'w', encoding="utf-8") as out_file:
#   with open('raw.tsv', 'r') as in_file:
    for line in file:
        print(line)
#         headline = line.split('\t')[1].strip().strip('"')
        out_file.write(line+"\n")

I tried ViewSonic M1+ , Nebula capsule max, capsule 2 projectors and finally I ended up with XGIMI Mogo pro.After I tried all these projectors, I feel XGIMI Mogo pro is best.
Below are the pros and consPros:1) Full HD2) Dual Wifi band support3) Good sound4) Quick auto focus5) Portable6) Build in Chromecast7) Less noise8) Android TV 9.0Cons:1) No official Netflix app.
But you can watch using XTV manager.2) No official Amazon prime app.
But you can cast using Chromecast.I hope they will add these primary apps soon.
Quick summary- this device provides high quality image in 1080p resolution with decent speakers and really works quite well.
My friend has a Nebula Capsule Max, but I think my pictures look brighter and clearer :) The device looks premium and easy to carry around.
I mainly use it to watch movies, and so far I'm very happy with it.However, there are some issues- it is not netflix certified.
The customer service side suggested several ways to watch it, including Roku.
I am able 

In [17]:
with open('xgimitext.txt', 'r', encoding="utf-8") as f:
    file = f.readlines()

Generate Embeddings for the Data

In [24]:
# Embedding extraction method

embed_fn = None

def generate_embeddings(text, module_url, random_projection_matrix=None):
  # Beam will run this function in different processes that need to
  # import hub and load embed_fn (if not previously loaded)
  global embed_fn
  if embed_fn is None:
    embed_fn = hub.load(module_url)
  embedding = embed_fn(text).numpy()
  if random_projection_matrix is not None:
    embedding = embedding.dot(random_projection_matrix)
  return text, embedding

In [25]:
# Convert to tf.Example method

def to_tf_example(entries):
  examples = []

  text_list, embedding_list = entries
  for i in range(len(text_list)):
    text = text_list[i]
    embedding = embedding_list[i]

    features = {
        'text': tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[text.encode('utf-8')])),
        'embedding': tf.train.Feature(
            float_list=tf.train.FloatList(value=embedding.tolist()))
    }

    example = tf.train.Example(
        features=tf.train.Features(
            feature=features)).SerializeToString(deterministic=True)

    examples.append(example)

  return examples

In [26]:
# Beam pipeline


def run_hub2emb(args):
  '''Runs the embedding generation pipeline'''

  options = beam.options.pipeline_options.PipelineOptions(**args)
  args = namedtuple("options", args.keys())(*args.values())

  with beam.Pipeline(args.runner, options=options) as pipeline:
    (
        pipeline
        | 'Read sentences from files' >> beam.io.ReadFromText(
            file_pattern=args.data_dir)
        | 'Batch elements' >> util.BatchElements(
            min_batch_size=args.batch_size, max_batch_size=args.batch_size)
        | 'Generate embeddings' >> beam.Map(
            generate_embeddings, args.module_url, args.random_projection_matrix)
        | 'Encode to tf example' >> beam.FlatMap(to_tf_example)
        | 'Write to TFRecords files' >> beam.io.WriteToTFRecord(
            file_path_prefix='{}/emb'.format(args.output_dir),
            file_name_suffix='.tfrecords')
    )

Generating Random Projection Weight Matrix

In [27]:
from sklearn import random_projection

def generate_random_projection_weights(original_dim, projected_dim):
  random_projection_matrix = None
#   X = np.random.rand( original_dim, projected_dim)
  X = np.random.normal( size= (original_dim, projected_dim))
  transformer = random_projection.GaussianRandomProjection(n_components=projected_dim)
  random_projection_matrix = transformer.fit_transform(X) #.T
#   random_projection_matrix = gaussian_random_matrix(
#       n_components=projected_dim, n_features=original_dim).T
  print("A Gaussian random weight matrix was creates with shape of {}".format(random_projection_matrix.shape))
  print('Storing random projection matrix to disk...')
  with open('random_projection_matrix', 'wb') as handle:
    pickle.dump(random_projection_matrix, 
                handle, protocol=pickle.HIGHEST_PROTOCOL)

  return random_projection_matrix

Set parameters

In [48]:
# module_url = 'https://tfhub.dev/google/nnlm-en-dim128/2'
#---download and unzip model in to load manually.
#---this was done to prevent connnectivity time out issues.


module_url = 'nnlm-en-dim128_2'
projected_dim = 64

Run pipeline

In [29]:
hub.load(module_url)(['']).shape[0]

1

In [30]:
hub.load(module_url)(['']).shape[1]

128

In [31]:


import tempfile

output_dir = tempfile.mkdtemp()
original_dim = hub.load(module_url)(['']).shape[1]
random_projection_matrix = None

if projected_dim:
  random_projection_matrix = generate_random_projection_weights(
      original_dim, projected_dim)

args = {
    'job_name': 'hub2emb-{}'.format(datetime.utcnow().strftime('%y%m%d-%H%M%S')),
    'runner': 'DirectRunner',
    'batch_size': 1024,
#     'data_dir': 'text.txt',
    'data_dir':'xgimitext.txt',
    'output_dir': output_dir,
    'module_url': module_url,
    'random_projection_matrix': random_projection_matrix,
}

print("Pipeline args are set.")
args

A Gaussian random weight matrix was creates with shape of (128, 64)
Storing random projection matrix to disk...
Pipeline args are set.


{'job_name': 'hub2emb-210714-125346',
 'runner': 'DirectRunner',
 'batch_size': 1024,
 'data_dir': 'xgimitext.txt',
 'output_dir': 'C:\\Users\\MYKie\\AppData\\Local\\Temp\\tmpkg7v4q2u',
 'module_url': 'nnlm-en-dim128_2',
 'random_projection_matrix': array([[ 3.01817674,  1.20722013, -0.13466821, ...,  0.62348606,
          0.27182485,  1.59763779],
        [-0.59482516,  0.9008925 ,  1.26315861, ..., -0.17842746,
          0.44461179,  0.25853739],
        [ 0.55093765,  0.93692367,  0.08765043, ..., -1.48875097,
         -1.6852731 ,  0.15129098],
        ...,
        [-0.15799701,  1.88902363, -0.18252194, ...,  0.79365805,
          0.2255481 , -0.46034259],
        [-1.41377211, -0.88582841,  1.14891645, ..., -0.47096394,
          0.56808247, -0.27023318],
        [ 0.49123434,  0.33759877,  0.94678226, ..., -1.65712772,
         -0.1407319 , -0.40070328]])}

In [32]:
print("Running pipeline...")
%time run_hub2emb(args)
print("Pipeline is done.")

Running pipeline...


         0.27182485,  1.59763779],
       [-0.59482516,  0.9008925 ,  1.26315861, ..., -0.17842746,
         0.44461179,  0.25853739],
       [ 0.55093765,  0.93692367,  0.08765043, ..., -1.48875097,
        -1.6852731 ,  0.15129098],
       ...,
       [-0.15799701,  1.88902363, -0.18252194, ...,  0.79365805,
         0.2255481 , -0.46034259],
       [-1.41377211, -0.88582841,  1.14891645, ..., -0.47096394,
         0.56808247, -0.27023318],
       [ 0.49123434,  0.33759877,  0.94678226, ..., -1.65712772,
        -0.1407319 , -0.40070328]])}


Wall time: 2.04 s
Pipeline is done.


In [33]:
ls {output_dir}

 Volume in drive C has no label.
 Volume Serial Number is 063B-D951

 Directory of C:\Users\MYKie\AppData\Local\Temp\tmpkg7v4q2u

07/14/2021  08:53 PM    <DIR>          .
07/14/2021  08:53 PM    <DIR>          ..
07/14/2021  08:53 PM           438,071 emb-00000-of-00001.tfrecords
               1 File(s)        438,071 bytes
               2 Dir(s)  57,575,550,976 bytes free


Read some of the generated embeddings...

In [34]:
embed_file = os.path.join(output_dir, 'emb-00000-of-00001.tfrecords')
sample = 5

# Create a description of the features.
feature_description = {
    'text': tf.io.FixedLenFeature([], tf.string),
    'embedding': tf.io.FixedLenFeature([projected_dim], tf.float32)
}

def _parse_example(example):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example, feature_description)

dataset = tf.data.TFRecordDataset(embed_file)
for record in dataset.take(sample).map(_parse_example):
  print("{}: {}".format(record['text'].numpy().decode('utf-8'), record['embedding'].numpy()[:10]))

I tried ViewSonic M1+ , Nebula capsule max, capsule 2 projectors and finally I ended up with XGIMI Mogo pro.After I tried all these projectors, I feel XGIMI Mogo pro is best.: [-1.3094931   0.5831628  -0.3201584   1.27804    -1.4095589  -0.4089551
 -0.5168101   1.590789   -0.56005496 -0.44856957]
Below are the pros and consPros:1) Full HD2) Dual Wifi band support3) Good sound4) Quick auto focus5) Portable6) Build in Chromecast7) Less noise8) Android TV 9.0Cons:1) No official Netflix app.: [-1.7083434   1.4371746   2.1689744  -0.26700366 -2.8384833  -1.894894
  1.0511308  -1.4468449  -1.1725187  -0.16940773]
But you can watch using XTV manager.2) No official Amazon prime app.: [ 2.0853274   1.111599    0.7450435  -0.3068385  -0.39272606 -1.3579074
  1.2916408  -1.3246647  -1.7642888   2.0982857 ]
But you can cast using Chromecast.I hope they will add these primary apps soon.: [ 1.8574227   0.8295457  -0.5771785   0.6033614  -0.80412626 -0.19141379
  0.0917436   0.01092126  0.09315198  1

Build the ANN Index for the Embeddings

ANNOY is similar to https://github.com/facebookresearch/faiss

In [35]:
def build_index(embedding_files_pattern, index_filename, vector_length, 
    metric='angular', num_trees=100):
  '''Builds an ANNOY index'''

  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  # Mapping between the item and its identifier in the index
  mapping = {}

  embed_files = tf.io.gfile.glob(embedding_files_pattern)
  num_files = len(embed_files)
  print('Found {} embedding file(s).'.format(num_files))

  item_counter = 0
  for i, embed_file in enumerate(embed_files):
    print('Loading embeddings in file {} of {}...'.format(i+1, num_files))
    dataset = tf.data.TFRecordDataset(embed_file)
    for record in dataset.map(_parse_example):
      text = record['text'].numpy().decode("utf-8")
      embedding = record['embedding'].numpy()
      mapping[item_counter] = text
      annoy_index.add_item(item_counter, embedding)
      item_counter += 1
      if item_counter % 100000 == 0:
        print('{} items loaded to the index'.format(item_counter))

  print('A total of {} items added to the index'.format(item_counter))

  print('Building the index with {} trees...'.format(num_trees))
  annoy_index.build(n_trees=num_trees)
  print('Index is successfully built.')

  print('Saving index to disk...')
  annoy_index.save(index_filename)
  print('Index is saved to disk.')
  print("Index file size: {} GB".format(
    round(os.path.getsize(index_filename) / float(1024 ** 3), 2)))
  annoy_index.unload()

  print('Saving mapping to disk...')
  with open(index_filename + '.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('Mapping is saved to disk.')
  print("Mapping file size: {} MB".format(
    round(os.path.getsize(index_filename + '.mapping') / float(1024 ** 2), 2)))

In [36]:
embedding_files = "{}/emb-*.tfrecords".format(output_dir)
embedding_dimension = projected_dim
index_filename = "index"

!rm {index_filename}
!rm {index_filename}.mapping

%time build_index(embedding_files, index_filename, embedding_dimension)

'rm' is not recognized as an internal or external command,
operable program or batch file.


Found 1 embedding file(s).
Loading embeddings in file 1 of 1...


'rm' is not recognized as an internal or external command,
operable program or batch file.


A total of 1087 items added to the index
Building the index with 100 trees...
Index is successfully built.
Saving index to disk...
Index is saved to disk.
Index file size: 0.0 GB
Saving mapping to disk...
Mapping is saved to disk.
Mapping file size: 0.1 MB
Wall time: 166 ms


Use the Index for Similarity Matching

Now we can use the ANN index to find news headlines that are semantically close to an input query.

Load the index and the mapping files

In [37]:
index = annoy.AnnoyIndex(embedding_dimension)
index.load(index_filename, prefault=True)
print('Annoy index is loaded.')
with open(index_filename + '.mapping', 'rb') as handle:
  mapping = pickle.load(handle)
print('Mapping file is loaded.')

Annoy index is loaded.
Mapping file is loaded.


C:\Users\MYKie\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


Similarity matching method

In [38]:
def find_similar_items(embedding, num_matches=5):
  '''Finds similar items to a given embedding in the ANN index'''
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=False)
  items = [mapping[i] for i in ids]
  return items

Extract embedding from a given query

In [39]:
# Load the TF-Hub module
print("Loading the TF-Hub module...")
%time embed_fn = hub.load(module_url)
print("TF-Hub module is loaded.")

random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
  print("Loading random projection matrix...")
  with open('random_projection_matrix', 'rb') as handle:
    random_projection_matrix = pickle.load(handle)
  print('random projection matrix is loaded.')

def extract_embeddings(query):
  '''Generates the embedding for the query'''
  query_embedding =  embed_fn([query])[0].numpy()
  if random_projection_matrix is not None:
    query_embedding = query_embedding.dot(random_projection_matrix)
  return query_embedding

Loading the TF-Hub module...
Wall time: 910 ms
TF-Hub module is loaded.
Loading random projection matrix...
random projection matrix is loaded.


In [40]:
extract_embeddings("Hello Machine Learning!")[:10]

array([-2.81048277,  1.65433781, -0.08270177,  0.46190737, -1.99411214,
        0.71846448, -1.43129142, -1.27770487, -0.97586752, -1.55146851])

Enter a query to find the most similar items

In [46]:
query = "the screen is very clear"

# query = "g7 warns of increasing global economic crisis"

# query = "business opportunities"

print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 10)

print("")
print("Results:")
print("=========")
items=set(items)
for item in items:
  print(item)

Generating embedding for the query...
Wall time: 1 ms

Finding relevant items in the index...
Wall time: 0 ns

Results:
It's quiet, the brightness is adjustable by color, the remote is great.
The picture quality is phenomenal and the sound from the built-in speaker is high-quality (for being built into the projector).
The screen pixelation that occurs on YouTube, Xgimi acknowledged this was a problem and said it will be in a patch in the future.
It comes extremely nicely packed and the quality of the product can be gauged at the first glance itself.
(light is on top which is very close to the part of wall I projected) There is a small light in a far corner in both cases.
The picture is best in a dark room, but even with a little light coming in the image is viewable.
The luxury of having a big screen on the go is something to appreciate.
The 300 ANSI lumens (how much light you are getting from a bulb/brightness) is surprisingly sufficient to enjoy a good movie.Its portability is defini

In [47]:
query = "connectivity"

# query = "g7 warns of increasing global economic crisis"

# query = "business opportunities"

print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 10)

print("")
print("Results:")
print("=========")
# print(type(items))
items=set(items)
for item in items:
  print(item)

Generating embedding for the query...
Wall time: 2 ms

Finding relevant items in the index...
Wall time: 995 µs

Results:
Also it needs to build in smart TV futures to stay ahead of competition.
It has auto focus.
Very light weight with amazing picture quality and built in android tv to access great variety  of entertainment on your finger tips.
We can stream content from any offer streaming platforms or devices connected via USB port or HDMI port.It is a simply amazing projector that I have ever seen which comes under well within the budget compared to any premium wired projectors.
It also supports HDMI input to project from external source.
I use it to connect computer for kids' screen time trying to reduce damage caused by regular monitor.I did not use projector screen and the definition is clear for kids to see.It is small I can move it to living room anytime to have a movie or game.It also support Android TV which is good to watch more programs.
You can connect a sound bar via blu